<a href="https://colab.research.google.com/github/mmistroni/OpenBB_Experiments/blob/main/OpenBBSamples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing the OpenBB Platform in Google Colab

This notebook will install the OpenBB Platform, fetch some data and prepare it for display as a bar chart.

Sign up for a free account here: https://my.openbb.co

In [1]:
# Install the OpenBB Platform with all available extensions.
# Messages indicating package version conflicts at the end of installation can be safely ignored.

!pip install openbb
!pip install pandas_ta
!pip install openbb-charting
# There is also a nightly distribution available, openbb-nightly

  Using cached pandas_ta-0.3.14b0-py3-none-any.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.4/259.4 kB 26.0 MB/s eta 0:00:00


In [5]:
!pip install openbb-finviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00


In [1]:
# Before running this cell, restart the runtime by selecting, "Restart runtime", from the "Runtime" menu.

# Import statements - for many scenarios, the only import needed will be `from openbb import obb`
from typing import Literal
from IPython.display import display
from IPython.display import clear_output
import ipywidgets as widgets
import pandas as pd
import pandas_ta as ta
from datetime import datetime , date
from plotly import graph_objects as go

from openbb import obb

Extensions to add: finviz@1.3.0

Building...


Logging in to OBB

In [2]:
# Login to OpenBB Hub to retrieve stored API keys.
# https://my.openbb.co/app/platform/pat
# https://my.openbb.co/app/platform/api-keys
from google.colab import userdata

MY_PAT = userdata.get('PAT_KEY')


obb.account.login(pat=MY_PAT)
# Setting preferences to always return a dataframe
obb.user.preferences.output_type = "dataframe"

### Defining Tickers for Analysis


In [3]:
# Set the output preference, if desired. The examples below use Pandas DataFrames.
from datetime import timedelta
from dateutil.relativedelta import relativedelta
TICKER = 'DIS'
TICKER2 = '^GSPC' # S&P500
SECTOR_TICKER = 'XLY'
INDUSTRY_TICKER = 'XME'
startDate = date.today() - relativedelta(years=5)

### Return Comparison

In [26]:
desc_filters = {
        'Market Cap.': '+Small (over $300mln)',
        'Average Volume': 'Over 200K',
        #'Relative Volume': 'Over 1',
    }
fund_filters = {
    'Return on Equity': 'Positive (>0%)',
    'InstitutionalOwnership': 'Under 60%',
    'Current Ratio' :  'Over 1.5',
    'Debt/Equity'   : 'Over 0.3'
}

desc_filters.update(fund_filters)

obb.equity.screener(provider='finviz', industry='semiconductors',
                    filters_dict=desc_filters
                    )

,symbol,name,country,sector,industry,market_cap,price,change_percent,volume,price_to_earnings
0,HIMX,Himax Technologies ADR,Taiwan,Technology,Semiconductors,1.080000e+09,6.32,0.0276,1127546,13.82


In [ ]:
44import matplotlib.pyplot as plt
spy = obb.equity.price.historical(symbol = TICKER2, provider='fmp')[['close']].rename(columns={'close' : TICKER2})
dis = obb.equity.price.historical(symbol = TICKER, provider='fmp')[['close']].rename(columns={'close' : TICKER})
sector = obb.equity.price.historical(symbol = SECTOR_TICKER, provider='fmp')[['close']].rename(columns={'close' : SECTOR_TICKER})
industry = obb.equity.price.historical(symbol = INDUSTRY_TICKER, provider='fmp')[['close']].rename(columns={'close' : INDUSTRY_TICKER})

one = pd.merge(spy, dis, left_index=True, right_index=True)
two = pd.merge(one, sector,left_index=True, right_index=True)
result = pd.merge(two, industry,left_index=True, right_index=True)
histpriceconcat = result/result.iloc[0]


histpriceconcat

for i, col in enumerate([TICKER2, TICKER, SECTOR_TICKER, INDUSTRY_TICKER]):
    histpriceconcat[col].plot()

plt.title('Price Evolution Comparison')
plt.xticks(rotation=70)

legendCols = [TICKER2, TICKER, f'{SECTOR_TICKER}(Sector)', f'{INDUSTRY_TICKER}(Industry)']

plt.legend(legendCols)
plt.show()



### Overview of the company

In [ ]:
obb.equity.profile(symbol=TICKER, provider='fmp')

### PEERS


In [ ]:
obb.equity.compare.peers(symbol=TICKER)

### Fetching some ratios


### Ratios Overview

In [ ]:
liq_ratios = ['current_ratio', 'quick_ratio', 'operating_cash_flow_per_share',  'days_of_sales_outstanding' ]
solvency_ratios = ['interest_coverage', 'debt_ratio',  'debt_equity_ratio'] #missing shareholder equity ratio
profitability_ratios = ['gross_profit_margin', 'operating_profit_margin',
                        'pretax_profit_margin',
                        'net_profit_margin',
                        'return_on_assets',
                        'return_on_equity',
                        'return_on_capital_employed',
                         ]

### Quarterly

In [ ]:
annual_ratios = obb.equity.fundamental.ratios(symbol=TICKER, period="annual", limit=5)
quarterly_ratios = obb.equity.fundamental.ratios(symbol=TICKER, period="quarter", limit=12)
quarterly_ratios['key'] = quarterly_ratios.apply(lambda row: f"{row['fiscal_period']}_{row['fiscal_year']}", axis=1)
# Income statement. we need this for trajectory of fundamentals
annual_income = obb.equity.fundamental.income(symbol=TICKER, period="annual", limit=5)
quarterly_income = obb.equity.fundamental.income(symbol=TICKER, period="quarter", limit=12)
quarterly_income['key'] = quarterly_ratios.apply(lambda row: f"{row['fiscal_period']}_{row['fiscal_year']}", axis=1)

quarterly_income

### Annually

In [ ]:
annual_liquidity = annual_ratios[liq_ratios + ['fiscal_year'] ].set_index('fiscal_year')
annual_solvency = annual_ratios[solvency_ratios + ['fiscal_year'] ].set_index('fiscal_year')
annual_profitability = annual_ratios[profitability_ratios + ['fiscal_year'] ].set_index('fiscal_year')
quarterly_liquidity = quarterly_ratios[liq_ratios + ['key'] ].set_index('key')
quarterly_solvency = quarterly_ratios[solvency_ratios + ['key'] ].set_index('key')
quarterly_profitability = quarterly_ratios[profitability_ratios + ['key'] ].set_index('key')

#annual_income = annual_income.set_index('fiscal_year')
#quarterly_income = quarterly_income.set_index('key')


annual_liquidity

### Some bar charts with OBB

In [ ]:
from openbb_charting.charts.generic_charts import bar_chart

bar_chart([annual_liquidity], x='fiscal_year', y=liq_ratios, title='Liquidity Ratios annual', normalize=True).show()

###Dividends Over Time

In [ ]:
df = obb.equity.fundamental.dividends(TICKER)
df['cob'] = pd.to_datetime(df.ex_dividend_date)
df = df.set_index(df.cob)
df[df.index > startDate.strftime('%Y-%m-%d')].plot(x='cob')


### Estimates

In [ ]:
obb.equity.estimates.consensus(symbol=TICKER)

### Splits

In [ ]:
obb.equity.fundamental.historical_splits(TICKER)

### Financial Statement Growth

In [ ]:
obb.equity.fundamental.balance_growth(symbol=TICKER, limit=10)

In [ ]:
obb.equity.fundamental.income_growth(symbol=TICKER, limit=10)

In [ ]:
obb.equity.fundamental.cash_growth(symbol=TICKER, limit=10)

### Insider Tradings

In [ ]:
obb.equity.ownership.insider_trading(symbol=TICKER, limit=500)

### 13F  - not Disney related -

In [ ]:
obb.equity.discovery.filings(limit=100, form_type='13F')

### Equity discovery

### Most Active

In [ ]:
obb.equity.discovery.active(sort="desc")

### Gainers

In [ ]:
obb.equity.discovery.gainers(sort="desc")

### Undervalued large caps

In [ ]:
obb.equity.discovery.undervalued_large_caps(sort="desc")

### Gainers

In [ ]:
obb.equity.discovery.gainers(limit=5)

In [ ]:
obb.equity.discovery.undervalued_growth(sort="desc")